### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [1]:
#Importing Libraies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression

#Function for removing punctation
def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# Importing Dataset
baby_df = pd.read_csv('amazon_baby.csv')
baby_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [2]:
#a)
# Removing punctuation from reviews by using the given function
baby_df['review'] = remove_punctuation(baby_df['review'].str)

#short test: 
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'
remove_punctuation(baby_df["review"][4]) == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

True

In [3]:
#b)
# Replacing all missing (nan) revies with empty "" string
baby_df['review'] = baby_df['review'].replace(np.nan, '', regex=True)

#short test:
baby_df["review"][38] == baby_df["review"][38]

True

In [4]:
#c)
# Droping all the entries with rating = 3, as they have neutral sentiment
baby_df = baby_df[baby_df.rating != 3]

#short test:
sum(baby_df["rating"] == 3)

0

In [5]:
#d) 
# Setting all positive (>=4) ratings to 1 and negative (<=2) to -1
baby_df.loc[baby_df['rating'] <= 2, 'rating'] = -1
baby_df.loc[baby_df['rating'] >= 4, 'rating'] = 1

#short test:
sum(baby_df["rating"]**2 != 1)

0

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names())
print(X_train_example.todense())

['adore', 'and', 'apples', 'bananas', 'dislike', 'hate', 'like', 'oranges', 'they', 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [7]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [14]:
#a)
# Importing train_test_split
from sklearn.model_selection import train_test_split

# Spliting data into training and tests sets
train_reviews, test_reviews, train_ratings, test_ratings = train_test_split(baby_df['review'], baby_df['rating'])

24991     1
146302   -1
2377      1
3009      1
175783    1
         ..
75140    -1
97040     1
138785    1
134998    1
179379    1
Name: rating, Length: 41688, dtype: int64

In [15]:
#b)
# Transforming reviewes into vectors using CountVectorizer
vectorizer = CountVectorizer()

# Train reviewes as vector
vector_train_reviews = vectorizer.fit_transform(train_reviews)

# Test reviewes as vector
vector_test_reviews = vectorizer.transform(test_reviews)

# CountVectorizer is a great tool provided by the scikit-learn library in Python. 
# It is used to transform a given text into a vector on the basis of the frequency (count) of each
# word that occurs in the entire text.

<125064x117040 sparse matrix of type '<class 'numpy.int64'>'
	with 6654700 stored elements in Compressed Sparse Row format>

## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [10]:
#a)
# Training LogisticRegression on training sets
model = LogisticRegression()
model.fit(vector_train_reviews, train_ratings)

# Logistic regression is an example of supervised learning. 
# It is used to calculate or predict the probability of a binary (yes/no) event occurring. 

/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [11]:
#b)
# Creating sorted words set 
words = sorted(list(zip(model.coef_[0], vectorizer.get_feature_names())), key=lambda x: x[0])

# Creating list for top 10 positive and negative words
top_10_positive = [word[1] for word in words[-10:]]
top_10_negative = [word[1] for word in words[:10]]

# Printing top 10 most positive words
print('Top 10 most positive words:', top_10_positive)

# Printing top 10 most negative words
print('Top 10 most negative words:', top_10_negative)

Top 10 most positive words: ['satisfied', 'complaint', 'lifesaver', 'worry', 'loves', 'complaints', 'amazing', 'perfect', 'pleased', 'excellent']
Top 10 most negative words: ['disappointing', 'worst', 'poorly', 'concept', 'disappointed', 'dissapointed', 'worthless', 'shame', 'useless', 'unusable']


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [12]:
#a)
# Importing time library
import time
start = time.time()
predictions = model.predict(vector_test_reviews)
predictions_time = time.time() - start
print(predictions)

# predict() - Python predict() function enables us to predict the labels of the data values on the basis of 
# the trained model. The predict() function accepts only a single argument which is usually 
# the data to be tested.

[-1  1  1 ...  1  1  1]


In [13]:
#b)
probability_predictions = model.predict_proba(vector_test_reviews)
print(probability_predictions)

# predict_proba() - The method accepts a single argument that corresponds to the data over which the 
# probabilities will be computed and returns an array of lists containing the class probabilities for 
#the input data points.

[[0.80392563 0.19607437]
 [0.00138226 0.99861774]
 [0.006047   0.993953  ]
 ...
 [0.00125238 0.99874762]
 [0.08938726 0.91061274]
 [0.01940839 0.98059161]]


In [14]:
#c) 
# Creating sorted reviews set 
reviews = sorted(list(zip(probability_predictions, test_reviews)), key=lambda x: x[0][1])

# Creating list for top 10 positive and negative words
top_5_positive = [review[1] for review in reviews[-5:]]
top_5_negative = [review[1] for review in reviews[:5]]

# Printing top 10 most positive words
print('Top 5 most positive reviews:\n')
for i in range(5):
    print('{}. {}\n'.format(i + 1, top_5_positive[i]))
    
# Printing top 10 most negative words
print('Top 5 most negative reviews:\n')
for i in range(5):
    print('{}. {}\n'.format(i + 1, top_5_negative[i]))

Top 5 most positive reviews:

1. We bought this stroller after selling our beloved BOB rev on craigslist We used the BOB for 9 months for my son but it just wasnt practical I dont jogrun it didnt have a big basket and was very bulky to take into stores quickly However I did love how it unfolded easily but it was heavy to fold up and lift into my small trunk myself Overall I didnt realize what Id need in a stroller until AFTER I had my son Live  learn We did love how easily the BOB would go over pretty much anything Nevertheless we sold it and after extensive research on strollers we decided it was between the uppababy brand because of the large baskets OR the city mini GT because of its easy fold up design After looking over both strollers I decided on the uppababy cruz because of a few main factors It SITS UP I cant tell you how much my son hates being reclined when he is just riding in the stroller and not napping The BOB and the City Mini had a slight recline and he always tried sit

In [15]:
#d) 
# Importing accuracy_score
from sklearn.metrics import accuracy_score

# Calculating the accuracy of predictions
predictions_accuracy = accuracy_score(test_ratings, predictions)
print(predictions_accuracy)

# The accuracy_score method says its return value depends on the setting for the normalize parameter: 
# If False, return the number of correctly classified samples. 
# Otherwise, return the fraction of correctly classified samples.

0.9328104010746497


## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [16]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']

In [17]:
#a)
# I decided to delete comments in this section
#2a)---------------------------------------------------------------------------------------------------------
train_reviews, test_reviews, train_ratings, test_ratings = train_test_split(baby_df['review'], baby_df['rating'])

#2b)---------------------------------------------------------------------------------------------------------
vectorizer = CountVectorizer(vocabulary=significant_words)
vector_train_reviews = vectorizer.fit_transform(train_reviews)
vector_test_reviews = vectorizer.transform(test_reviews)

#3a)---------------------------------------------------------------------------------------------------------
model = LogisticRegression()
model.fit(vector_train_reviews, train_ratings)

#3b)---------------------------------------------------------------------------------------------------------
words = sorted(list(zip(model.coef_[0], vectorizer.get_feature_names())), key=lambda x: x[0])
top_10_positive = [word[1] for word in words[-10:]]
top_10_negative = [word[1] for word in words[:10]]
print('Top 10 most positive words:', top_10_positive)
print('Top 10 most negative words:', top_10_negative)

#4a)---------------------------------------------------------------------------------------------------------
start = time.time()
predictions_dictionary = model.predict(vector_test_reviews)
predictions_time_dictionary = time.time() - start
print(predictions)

#4b)---------------------------------------------------------------------------------------------------------
probability_predictions = model.predict_proba(vector_test_reviews)
print(probability_predictions)

#4c)---------------------------------------------------------------------------------------------------------
reviews = sorted(list(zip(probability_predictions, test_reviews)), key=lambda x: x[0][1])
top_5_positive = [review[1] for review in reviews[-5:]]
top_5_negative = [review[1] for review in reviews[:5]]
print('Top 5 most positive reviews:\n')
for i in range(5):
    print('{}. {}\n'.format(i + 1, top_5_positive[i]))
print('Top 5 most negative reviews:\n')
for i in range(5):
    print('{}. {}\n'.format(i + 1, top_5_negative[i]))

#4d)---------------------------------------------------------------------------------------------------------
predictions_accuracy_dictionary = accuracy_score(test_ratings, predictions_dictionary)
print(predictions_accuracy_dictionary)

Top 10 most positive words: ['car', 'old', 'able', 'well', 'little', 'great', 'easy', 'love', 'perfect', 'loves']
Top 10 most negative words: ['disappointed', 'waste', 'return', 'broke', 'money', 'work', 'even', 'would', 'product', 'less']
[-1  1  1 ...  1  1  1]
[[0.06733396 0.93266604]
 [0.04550928 0.95449072]
 [0.21389392 0.78610608]
 ...
 [0.14243191 0.85756809]
 [0.10898125 0.89101875]
 [0.01694395 0.98305605]]
Top 5 most positive reviews:

1. Background Ive been using Grovia diapers for four years when I bought them they were called grobaby I have 6 shells and 12 inserts I purchased them for my oldest daughter and used them only part time for 25 years until she was potty trained She still needed pullups at night for a long time so I used these until she was ready to go without She was probably 30lbs when she stopped wearing them and they fit well They were stored for a year until I had my other daughter who is now 55 months and exclusively CDd I didnt use them for either girl unt

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [18]:
#b)
# Importing Fabs
from math import fabs

max_impact = max(max(model.coef_[0]), fabs(min(model.coef_[0])))

# Printing Maximum relative impact
print('Maximum relative impact = {0:.5f}'.format(max_impact))

dict_words_impact = [val for val in words if any(s in val for s in significant_words)]

# Printing word with the biggest impact
for word_impact in dict_words_impact:
    print('Word {0} has impact {1:.5f}'.format(word_impact[1], fabs(word_impact[0])))

Maximum relative impact = 2.37319
Word disappointed has impact 2.37319
Word waste has impact 2.03966
Word return has impact 2.00976
Word broke has impact 1.60824
Word money has impact 0.87824
Word work has impact 0.63352
Word even has impact 0.52719
Word would has impact 0.34040
Word product has impact 0.31608
Word less has impact 0.19285
Word car has impact 0.05335
Word old has impact 0.08132
Word able has impact 0.21528
Word well has impact 0.49362
Word little has impact 0.51398
Word great has impact 0.93617
Word easy has impact 1.13813
Word love has impact 1.32677
Word perfect has impact 1.43606
Word loves has impact 1.74165


In [37]:
#c)
# Just printing the results

print("Accuracy with all the words {0:.2f}%.".format((predictions_accuracy)*100))
print("Accuracy with limited dictionary {0:.2f}%.".format((predictions_accuracy_dictionary)*100))
print("Using all the words is better by {0:.2f}%.".format((predictions_accuracy-predictions_accuracy_dictionary)*100))
print("Time of evaluation on all the words: {0:.5f}ms".format(predictions_time * 1000))
print("Time of evaluation on limited dictionary: {0:.5f}ms".format(predictions_time_dictionary * 1000))
print("Using limited dictionary is {0:.5f}ms faster.".format((predictions_accuracy-predictions_accuracy_dictionary)*1000))
print("Using limited dictionary is x{0:.2f} times faster.".format((predictions_time/predictions_time_dictionary)))

# The results meet my expectations. Using all the words, the result is more accurate than the one with a limited 
# dictionary. All words, however, are associated with a longer time of evaluation.

Accuracy with all the words 93.28%.
Accuracy with limited dictionary 86.90%.
Using all the words is better by 6.39%.
Time of evaluation on all the words: 9.06610ms
Time of evaluation on limited dictionary: 1.99723ms
Using limited dictionary is 63.85531ms faster.
Using limited dictionary is x4.54 times faster.
